## Section 0: Load the dataset and import libraries
The Titanic dataset is loaded into a pandas DataFrame in this section along with the necessary Python packages.
To comprehend the dataset's structure, we also look at the initial few rows and the most fundamental metadata.

In [1]:
import pandas as pd
import numpy as np

df= pd.read_csv("Downloads/titanic.csv")

display(df.head())
display(df.info())
display(df.describe(include="all"))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


## Section 1: Variable Types and Names
Every variable is categorized as either text, ordinal, categorical, or numerical.
Pandas dtype and known variable semantics are used by a helper function to determine the type.
A Markdown table is printed as the outcome.

In [2]:
def classify_aspects(series: pd.Series) -> str:
    if pd.api.types.is_numeric_dtype(series):
        if series.name == "Pclass":
            return "Numerical (ordinal categorical)"
        elif series.name == "Survived":
            return "Numerical (binary categorical)"
        elif series.name in ["SibSp", "Parch", "PassengerId"]:
            return "Numerical (discrete count / identifier)"
        else:
            return "Numerical (continuous)"
    else:
        if series.name in ["Sex", "Embarked"]:
            return "Categorical nominal"
        else:
            return "Text (categorical nominal)"

var_info = [{"Variable": col, "Type": classify_aspects(df[col])} for col in df.columns]
var_df = pd.DataFrame(var_info)

print(var_df.to_markdown(index=False))

| Variable    | Type                                    |
|:------------|:----------------------------------------|
| PassengerId | Numerical (discrete count / identifier) |
| Survived    | Numerical (binary categorical)          |
| Pclass      | Numerical (ordinal categorical)         |
| Name        | Text (categorical nominal)              |
| Sex         | Categorical nominal                     |
| Age         | Numerical (continuous)                  |
| SibSp       | Numerical (discrete count / identifier) |
| Parch       | Numerical (discrete count / identifier) |
| Ticket      | Text (categorical nominal)              |
| Fare        | Numerical (continuous)                  |
| Cabin       | Text (categorical nominal)              |
| Embarked    | Categorical nominal                     |


## Section 2.1: Impute Embarked Missing Values
We determine the Embarked column's mode, or most frequent value, and use it to fill in any missing entries.

In [3]:
official_embarked_mode = df["Embarked"].mode(dropna=True)[0]
df["Embarked_imputed"] = df["Embarked"].fillna(official_embarked_mode)

official_embarked_mode, df[["Embarked", "Embarked_imputed"]].head()

('S',
   Embarked Embarked_imputed
 0        S                S
 1        C                C
 2        S                S
 3        S                S
 4        S                S)

## Section 2.2: Mean-Based Age Imputation
We determine the global mean age and use it to fill in the missing Age data.

In [4]:
official_age_mean = df["Age"].mean(skipna=True)
df["Age_mean_imputed"] = df["Age"].fillna(official_age_mean)

official_age_mean, df[["Age", "Age_mean_imputed"]].head()

(np.float64(29.69911764705882),
     Age  Age_mean_imputed
 0  22.0              22.0
 1  38.0              38.0
 2  26.0              26.0
 3  35.0              35.0
 4  35.0              35.0)

## Section 2.3: Groupwise Median Age Imputation
For every Sex × Pclass combination, we calculate the median Age and use it to fill in the blanks.

In [5]:
median_age_group = df.groupby(["Sex", "Pclass"])["Age"].median()

def impute_age_groupwise(row):
    if pd.isna(row["Age"]):
        return median_age_group.loc[(row["Sex"], row["Pclass"])]
    return row["Age"]

df["Age_group_median_imputed"] = df.apply(impute_age_groupwise, axis=1)

median_age_group, df[["Sex", "Pclass", "Age", "Age_group_median_imputed"]].head()

(Sex     Pclass
 female  1         35.0
         2         28.0
         3         21.5
 male    1         40.0
         2         30.0
         3         25.0
 Name: Age, dtype: float64,
       Sex  Pclass   Age  Age_group_median_imputed
 0    male       3  22.0                      22.0
 1  female       1  38.0                      38.0
 2  female       3  26.0                      26.0
 3  female       1  35.0                      35.0
 4    male       3  35.0                      35.0)

## Section 3: Creation of FamilySize and Summary Data
The definition of FamilySize is:  **SibSp + Parch + 1 = FamilySize**  
We collect the number of passengers who traveled alone and calculate the minimum, maximum, and mean.

In [6]:
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

family_minimum = df["FamilySize"].min()
family_maximum = df["FamilySize"].max()
family_mean = df["FamilySize"].mean()
traveling_alone = (df["FamilySize"] == 1).sum()

family_minimum, family_maximum, family_mean, traveling_alone

(1, 11, np.float64(1.904601571268238), np.int64(537))

## Section 4.1 : Construct Filtered DataFrames The dataset is filtered into:
Every woman, every man
Every traveler under the age of sixteen  

In [7]:
df_female = df[df["Sex"] == "female"]
df_male = df[df["Sex"] == "male"]
df_under16 = df[df["Age"] < 16]

len(df_female), len(df_male), len(df_under16)

(314, 577, 83)

## Section 4.2: Calculate Filtered Groups' Survival Rates
We calculate the average survival rate for passengers under 16, males, and females.

In [8]:
female_survival_rate = df_female["Survived"].mean()
male_survival_rate = df_male["Survived"].mean()
under16_survival_rate = df_under16["Survived"].mean()

female_survival_rate, male_survival_rate, under16_survival_rate

(np.float64(0.7420382165605095),
 np.float64(0.18890814558058924),
 np.float64(0.5903614457831325))

## Section 4.3: Survival Rate by Pclass and Sex
We calculate mean survival, group by Pclass and Sex, then provide the outcome as a Markdown table.

In [9]:
survival_sex_pclass = df.groupby(["Sex", "Pclass"])["Survived"].mean().unstack("Pclass")

print(survival_sex_pclass.to_markdown())
survival_sex_pclass

| Sex    |        1 |        2 |        3 |
|:-------|---------:|---------:|---------:|
| female | 0.968085 | 0.921053 | 0.5      |
| male   | 0.368852 | 0.157407 | 0.135447 |


Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


## Section 4.4: Interpretation
The observed survival patterns are summed up in two to three sentences.

In [10]:
print("""
Females in all classes had substantially better survival rates, which is consistent with the evacuation protocol's "women and children first" policy. The impact of socioeconomic level on access to lifeboats was demonstrated by the greater survival rates of first-class passengers compared to those in second and particularly third class.
""")


Females in all classes had substantially better survival rates, which is consistent with the evacuation protocol's "women and children first" policy. The impact of socioeconomic level on access to lifeboats was demonstrated by the greater survival rates of first-class passengers compared to those in second and particularly third class.

